In [1]:
import sys
sys.path.append("/private/home/gulordava/")

In [2]:
from syntactic_testsets.evaluate_utils import query_KenLM

## Lang

In [25]:
lang = "Hebrew"
short = lang.lower()[:2]
print(short)

he


In [26]:
data_path = "/private/home/gulordava/colorlessgreen/data/agreement/" + lang + "/generated.tab"
lm_file = "/private/home/gulordava/ngram_lms/" + lang + "/" + short + ".binary"

In [27]:
import pandas as pd
full_df = pd.read_csv(data_path,sep="\t")


In [28]:
to_test = open("temp.tmp", 'w')
for prefix in full_df.prefix + " " + full_df.form:
    to_test.write(prefix + "\n")
to_test.close()

In [29]:
result_probs, lines = query_KenLM(lm_file, "temp.tmp")

Perplexity including OOVs:	4560.932140738035
Perplexity excluding OOVs:	4445.762576857454
OOVs:	1220
Tokens:	152540
Name:query	VmPeak:2800352 kB	VmRSS:3824 kB	RSSMax:2777456 kB	user:0.056	sys:0.94	CPU:0.997141	real:2.71059


In [30]:
full_df["ngram"] = result_probs

In [31]:
import numpy as np

wide_data = full_df[["pattern","constr_id", "sent_id", "class", "ngram","type"]].pivot_table(columns=("class"),values=("ngram"),index=["pattern","constr_id", "sent_id","type"])
wide_data["acc"] = wide_data.correct > wide_data.wrong
wide_data = wide_data.reset_index()
for pattern in set(wide_data["pattern"]):
    acc_generated = np.mean(wide_data[wide_data.pattern == pattern][wide_data.type == "generated"]["acc"])
    acc_original = np.mean(wide_data[wide_data.pattern == pattern][wide_data.type == "original"]["acc"])
    if acc_generated > acc_original:
        print(pattern, acc_generated, acc_original)

wide_data.groupby(["type"]).agg({"acc":["mean","size"]})



L__NOUN_NOUN_PROPN_VERB 0.444444444444 0.333333333333
R__VERB_NOUN_PUNCT_CCONJ_VERB 0.583333333333 0.5
R__NOUN_ADJ_PUNCT_SCONJ_VERB 0.673611111111 0.5625
L__NOUN_PROPN_VERB 0.222222222222 0.0
R__NOUN_NOUN_PUNCT_SCONJ_VERB 0.587301587302 0.47619047619
L__NOUN_VERB_PUNCT_VERB 0.694444444444 0.5


/private/home/gulordava/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/private/home/gulordava/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


acc      
               mean  size
type                     
generated  0.616622  3357
original   0.721180   373

### (Macro average)

In [548]:

for pattern in set(wide_data.pattern):
    p = wide_data[wide_data.pattern == pattern]
    c_df = []
    for constr in set(p.constr_id):
        c = p[p.constr_id == constr]
        g = c[c.type == "generated"]
        o = c[c.type == "original"]
        if len(g) >0  and len(o) > 0:
            acc_o = np.mean(o.correct > o.wrong)
            acc_g = np.mean(g.correct > g.wrong)

            c_df.append((constr, acc_o, acc_g))
            
    
    _, acc_os, acc_gs = zip(*c_df)
    print(np.mean(acc_os), np.mean(acc_gs))

0.518518518519 0.456790123457
0.666666666667 0.62962962963
0.611111111111 0.679012345679
0.928571428571 0.52380952381
0.692307692308 0.547008547009
0.923076923077 0.581196581197
0.3 0.277777777778
0.555555555556 0.543209876543


In [ ]:
def macro_acc(data):
    wide_data = data[["pattern_id","constr_id", "sent_id", "class", "ngram"]].pivot_table(columns=("class"),values=("ngram"),index=["pattern_id","constr_id", "sent_id"])
    wide_data["acc"] = wide_data.correct > wide_data.wrong
    #acc = 1
    return np.mean(wide_data.groupby("constr_id").mean()["acc"])

In [ ]:
idx = [4,5,7,15,16,17,18,19,21,23,24,26,27]

In [ ]:
macro_acc(full_df[full_df.type == "generated"])

In [ ]:
macro_acc(full_df[full_df.type == "original"])

## Computing average ngram size

In [32]:
len(lines)

7465

In [33]:
len(full_df)

7460

In [34]:
p = ""
ns_o_all = []
ns_g_all = []
ns_o_tg = []
ns_g_tg = []

for i in range(len(full_df)):
    row = full_df.iloc[i]
    l = lines[i]
    ngram_size_all = [int(f.split()[1]) for f in l.split("\t")[:-1]]
    ngram_size_target = ngram_size_all[-1]
    if row.sent_id == 0:
        ns_o_all.extend(ngram_size_all)
        ns_o_tg.append(ngram_size_target)
    else:
        ns_g_all.extend(ngram_size_all)
        ns_g_tg.append(ngram_size_target)
    
    
print("All original/generated", np.mean(ns_o_all), np.mean(ns_g_all), sep="\t")
print("Target original/generated", np.mean(ns_o_tg), np.mean(ns_g_tg), sep="\t")
        

All original/generated	2.60423495477	1.98197194179
Target original/generated	2.59115281501	1.93550789395


In [35]:
%load_ext rpy2.ipython

In [ ]:
%Rpush full_df

In [ ]:
%%R
library(data.table)
full_df = data.table(full_df)

In [ ]:
%%R
print(cor(full_df$ngram, full_df$freq))
print(cor(full_df[sent_id == 0]$ngram, full_df[sent_id == 0]$freq))
cor(full_df[sent_id != 0]$ngram, full_df[sent_id != 0]$freq)